# Play by Play NFL Analysis of the 2015 season

By Adam Baca, Zoheb Allam, Scott Gozdzialski, Ethan Graham, 

### Business Understanding


Text Here for Business Understanding

### Data Meaning Type

In [111]:
import numpy as np
import pandas as pd
url = 'https://raw.githubusercontent.com/agbaca/DataMiningFall17/master/NFLPlaybyPlay2015.csv'
df = pd.read_csv(url,index_col=0,parse_dates=[0])

df.drop(['desc','PlayAttempted','sp','DefTwoPoint','PuntResult','Passer','Interceptor'],axis=1, inplace=True)
df.drop(['Rusher','Receiver','Returner','Tackler1','Tackler2','RecFumbPlayer','PenaltyType'],axis=1, inplace=True)
df.drop(['PenalizedPlayer','Season','time','TimeUnder','PlayTimeDiff','yrdln','PassAttempt'],axis=1, inplace=True)
df.drop(['RushAttempt','BlockingPlayer','GoalToGo','FirstDown','Safety','Sack','RunGap','Reception'],axis=1, inplace=True)
df.drop(['ReturnResult','PassOutcome'],axis=1, inplace=True)

The NFL 2015 dataset originally had over  65 different attributes per row.  Many of them are dulicates or are not need for our analysis. Our focus being on the team stats and not on individual stats we dropped any player specific attributes.

The variables we focused on are on 35 different columns that are most realated to the yards gained and ultimately to drive result.  

*Note some of the columns are kept at this point to give away of coombining the data for cluster analysis

The 35differnt rows are

In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46129 entries, 36 to 39949
Data columns (total 34 columns):
Date                  46129 non-null object
GameID                46129 non-null int64
Drive                 46129 non-null int64
qtr                   46129 non-null int64
down                  39006 non-null float64
TimeSecs              46102 non-null float64
SideofField           46063 non-null object
yrdline100            46021 non-null float64
ydstogo               46129 non-null int64
ydsnet                46129 non-null int64
posteam               42878 non-null object
DefensiveTeam         42878 non-null object
Yards.Gained          46129 non-null int64
Touchdown             46129 non-null int64
ExPointResult         1131 non-null object
TwoPointConv          89 non-null object
PlayType              46129 non-null object
PassLength            19291 non-null object
PassLocation          19291 non-null object
InterceptionThrown    46129 non-null int64
RunLocation        

The columns for this analysis are 


Column name          Description                                      Values
Date                 Date of game                                     Date
GameID               Unique game ID                                   Interger
Drvie                The way to id which drive it is in the game      Interger
qtr                  Quarter of the game                              1-4
down                 Which of the four downs                          1-4, Na
Timesecs             Total game time left in seconds                  3600 - 0
SideofField          Which side of the field the ball is on           Team Code
yrdline100           How far the ball is from the endzone             0-100
ydrtogo              Yards to go before a first down                  28-1
ydsnet               total yards gained in the drive                  0-100
Posteam              Team with the ball                               Team Code
DefensiveTeam        Team Defending                                   Team Code
Touchdown            If the posteam made a touchdown                  0, 1
ExPointResults       If the extra point was made                      0, 1
TwoPointConv         If the posteam made a2 point conversion          0, 1

PlayType             What type of play it was                         Pass, Run, 
                                                                      Kick off, Field Goal
                                                                      Punt, Sack, No Play,
                                                                      QB Kneel, Two minute warning
                                                                      Timout, Onside Kick, 
                                                                      End of Game, Quarter ended
                                                                      extra point
                                                                               
PassLength           Where the pass was thrown                        Short, Deep, NA          
PassLocation         Location of pass on field                        Left, Righ, Middle, NA 
InterceptionThrown   Was the ball intercepted                         0, 1
RunLocation          Location of the run                              Left, right, Middle, NA   
FieldGoalResult      If the field goal was made                       Good, No Good ,NA 
FieldGoalDistance    How far the field goal attemp was                50-0 
Fumble               If there was a fumble on the play                0 , 1
RecFumbTeam          Team that recover the fumbel                     Team code 
Challenge.Replay     If the play ws challenged by the coach           0, 1
ChalReplayResult     Challenge Result                                 Reversed, Upheld, NA 
Accepted.Penalty     If the Penaly was accepted by the team           0, 1
PenalizedTeam        Team that was penalized                          Team Code
Penalty.Yards        Yard of the penalty                              0 - 15
PosTeamScore         Possetion team score                             Integer
DefTeamScore         Defending team score                             Integer
ScoreDiff            The difference between posteam/defteam score     positive and negative integer
AbsScoreDiff         The absolute value of score diff                 Integer



The Team Codes are listed below
NE, PIT, NA, GB, CHI, MID
HOU, KC, CLE, NYJ, BUF, IND
MIA, WAS, CAR, JAC, STL, SEA
NO, ARI, SD, DET, TB, TEN
CIN, OAK, DEN, BAL, NYG, DAL
ATL, PHI, MIN, SF

### Data Quality

There are many missing values in the different fields, as can be seen above, because many of the fields are binary flags with 0 for failed, 1 for seccuess, and NA for not attemped.  As we progress with the analysis we might need to keep or remove some of these, therefore we are not looking to change any of theses at this point.

There are no outliers because every column has discrete ranges associated with them, ie there are only 4 down, there is only 100 yards on the field, there are only so many play types, etc.  The excpetions to this rule would be the fields marked with integer but they can only be an interger and have a top and bottme range.

### Simple Statistics

In [113]:
df.describe()

,GameID,Drive,qtr,down,TimeSecs,yrdline100,ydstogo,ydsnet,Yards.Gained,Touchdown,InterceptionThrown,FieldGoalDistance,Fumble,Challenge.Replay,Accepted.Penalty,Penalty.Yards,PosTeamScore,DefTeamScore,ScoreDiff,AbsScoreDiff
count,4.612900e+04,46129.000000,46129.000000,39006.000000,46102.000000,46021.000000,46129.000000,46129.000000,46129.000000,46129.000000,46129.000000,989.000000,46129.000000,46129.000000,46129.000000,46129.000000,42878.000000,42878.00000,42878.000000,42878.000000
mean,2.015164e+09,12.279607,2.583407,1.996949,1686.735847,49.411312,7.310152,26.672289,4.885538,0.029309,0.010145,38.166835,0.013592,0.008953,0.076633,0.652388,10.586408,11.65024,-1.063832,7.773100
std,2.183164e+05,7.144244,1.134256,1.003834,1065.494471,24.852971,4.920808,25.379980,8.965909,0.168673,0.100214,10.337203,0.115792,0.094198,0.266011,2.716825,9.495190,9.96655,10.716223,7.452965
min,2.015091e+09,1.000000,1.000000,1.000000,-747.000000,1.000000,0.000000,-48.000000,-34.000000,0.000000,0.000000,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,-41.000000,0.000000
25%,2.015101e+09,6.000000,2.000000,1.000000,759.000000,31.000000,3.000000,5.000000,0.000000,0.000000,0.000000,30.000000,0.000000,0.000000,0.000000,0.000000,3.000000,3.00000,-7.000000,3.000000
50%,2.015111e+09,12.000000,3.000000,2.000000,1800.000000,50.000000,9.000000,20.000000,1.000000,0.000000,0.000000,39.000000,0.000000,0.000000,0.000000,0.000000,9.000000,10.00000,0.000000,6.000000
75%,2.015121e+09,18.000000,4.000000,3.000000,2579.000000,71.000000,10.000000,45.000000,7.000000,0.000000,0.000000,47.000000,0.000000,0.000000,0.000000,0.000000,17.000000,17.00000,5.000000,11.000000
max,2.016010e+09,33.000000,5.000000,4.000000,3600.000000,99.000000,42.000000,99.000000,99.000000,1.000000,1.000000,66.000000,1.000000,1.000000,1.000000,55.000000,52.000000,51.00000,41.000000,41.000000


We start with a basic function giving the count, mena, std, min, 25%, 50% or median, 75%, and max.  we can see some attributes who have les the complete count, like down, that we would not expect.  For down it is beacuse kick offs are NA, TimeSecs is because negative numbers are used for overtime and all games end in zero throwing off the count function. Field Goal distance is because the non-FG plays are NA.  Also, at the end of the quarter and timeouts cause problems with the scores and a few other attributes, but we are keeping them in to see if they provide any insight.

We will be looking at a box plot of yards.gain, a count of playtype, touchdown, Interception, and downs. 

In [114]:
df['PlayType'].value_counts()

Pass                  18323
Run                   13109
No Play                2608
Kickoff                2565
Punt                   2443
Timeout                1859
Sack                   1191
Extra Point            1126
Field Goal              988
Quarter End             675
Two Minute Warning      481
QB Kneel                425
End of Game             211
Onside Kick              67
Spike                    52
Half End                  6
Name: PlayType, dtype: int64

In [106]:
df['Touchdown'].value_counts()

0    44777
1     1352
Name: Touchdown, dtype: int64

In [107]:
df['InterceptionThrown'].value_counts()

0    45661
1      468
Name: InterceptionThrown, dtype: int64

In [108]:
df['down'].value_counts()

1.0    15703
2.0    11728
3.0     7566
4.0     4009
Name: down, dtype: int64

### Visualize Atttibutes

### Explore Joint Attributes and Class

### New Features

### Exceptional Work